In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
def get_comparisons(df: pd.DataFrame):
  comparisons: dict[tuple[str, str], tuple[int, int]] = {}

  for col in df.columns:
    profile = df[col]
    voters_num = int(profile[0])
    preferences = profile[1:]

    for candidate1_index in range(1, len(preferences) + 1):
      for candidate2_index in range(candidate1_index + 1, len(preferences) + 1):
        reversed_candidates = False

        candidate1 = preferences[candidate1_index]
        candidate2 = preferences[candidate2_index]

        if (candidate1, candidate2) in comparisons:
          votes1, votes2 = comparisons[(candidate1, candidate2)]
        elif (candidate2, candidate1) in comparisons:
          reversed_candidates = True
          votes2, votes1 = comparisons[(candidate2, candidate1)]
        else:
          votes1, votes2 = 0, 0

        if candidate1_index < candidate2_index:
          votes1 += voters_num
        else:
          votes2 += voters_num

        if reversed_candidates:
          comparisons[(candidate2, candidate1)] = (votes2, votes1)
        else:
          comparisons[(candidate1, candidate2)] = (votes1, votes2)

  return comparisons

In [22]:
def get_candidates(df: pd.DataFrame) -> np.ndarray[str]:
  candidates = np.array([], dtype=str)

  for col in df.columns:
    profile = df[col]
    preferences = profile[1:]

    for candidate_index in range(1, len(preferences) + 1):
      candidate = preferences[candidate_index]

      if candidate not in candidates:
        candidates = np.append(candidates, candidate)

  return candidates

In [23]:
def get_candidates_num(df: pd.DataFrame) -> int:
  candidates = set()

  for col in df.columns:
    profile = df[col]
    preferences = profile[1:]

    for candidate_index in range(1, len(preferences) + 1):
      candidates.add(preferences[candidate_index])

  return len(candidates)

In [24]:
def copelands_method(df: pd.DataFrame):
  comparisons = get_comparisons(df)
  stats: dict[str, dict] = {}

  for key, value in comparisons.items():
    candidate1, candidate2 = key
    votes1, votes2 = value

    if candidate1 not in stats:
      stats[candidate1] = {'wins': 0, 'losses': 0, 'ties': 0, 'net': 0, 'comparisons': {}}
    if candidate2 not in stats:
      stats[candidate2] = {'wins': 0, 'losses': 0, 'ties': 0, 'net': 0, 'comparisons': {}}

    winner = None
    loser = None

    if votes1 > votes2:
      winner = candidate1
      loser = candidate2
    elif votes1 < votes2:
      winner = candidate2
      loser = candidate1

    if winner and loser:
      stats[winner]['wins'] += 1
      stats[loser]['losses'] += 1
      stats[winner]['net'] += 1
      stats[loser]['net'] -= 1
      stats[winner]['comparisons'][loser] = 'win'
      stats[loser]['comparisons'][winner] = 'loss'

      continue

    stats[candidate1]['ties'] += 1
    stats[candidate2]['ties'] += 1
    stats[candidate1]['comparisons'][candidate2] = 'tie'
    stats[candidate2]['comparisons'][candidate1] = 'tie'

  winner = max(stats, key=lambda key: stats[key]['net'])

  return winner, stats, comparisons


In [25]:
def borda_count(df):
  candidate_num = get_candidates_num(df)
  candidate_to_points = {}

  for col in df.columns:
    profile = df[col]
    voters = int(profile[0])
    preferences = profile[1:].to_numpy()

    for candidate_index, candidate in enumerate(preferences):
      if candidate not in candidate_to_points:
        candidate_to_points[candidate] = 0

      candidate_to_points[candidate] += (candidate_num - candidate_index - 1) * voters

  winner = max(candidate_to_points, key=candidate_to_points.get)

  return winner, candidate_to_points

In [26]:
def parallel_exclusion_method(df: pd.DataFrame) -> tuple[str, dict[tuple[str, str], str]]:
  candidates = get_candidates(df)

  if len(candidates) == 1:
    return candidates[0]

  if not len(candidates) % 2 == 0:
    raise Exception('Number of voters is not even')
  
  tree_of_comparisons: dict[tuple[str, str], str] = {}
  comparisons = get_comparisons(df)

  def divide_and_conquer(start: int, end: int) -> str:
    if start == end:
      return candidates[start]

    middle = (start + end) // 2

    left_winner = divide_and_conquer(start, middle)
    right_winner = divide_and_conquer(middle + 1, end)

    left_winner_points = 0
    right_winner_points = 0

    if (left_winner, right_winner) in comparisons:
      left_winner_points, right_winner_points = comparisons[(left_winner, right_winner)]
    elif (right_winner, left_winner) in comparisons:
      right_winner_points, left_winner_points = comparisons[(right_winner, left_winner)]
    else:
      raise Exception('No comparison found')

    if left_winner_points > right_winner_points or left_winner_points == right_winner_points:
      tree_of_comparisons[(left_winner, right_winner)] = left_winner

      return left_winner

    tree_of_comparisons[(left_winner, right_winner)] = right_winner

    return right_winner

  return divide_and_conquer(0, len(candidates) - 1), tree_of_comparisons

In [27]:
df = pd.read_csv('data.csv')

df

,Profile1,Profile2,Profile3
0,5,4,2
1,a,c,b
2,b,a,a
3,c,b,c
4,d,d,d


In [28]:
borda_method_results = borda_count(df)

print("Borda method results")
print("Winner:", borda_method_results[0])
print("Scores:")
pd.DataFrame(borda_method_results[1].items(), columns=['Candidate', 'Score'])

Borda method results
Winner: a
Scores:


,Candidate,Score
0,a,27
1,b,20
2,c,19
3,d,0


In [29]:
copelands_method_results = copelands_method(df)

print("Copeland's method results")
print("Winner:", copelands_method_results[0])
print("Stats:")

data_to_plot = deepcopy(copelands_method_results[1])

for key in data_to_plot.keys():
  del data_to_plot[key]['comparisons']

pd.DataFrame(data_to_plot).T

Copeland's method results
Winner: a
Stats:


,wins,losses,ties,net
a,3,0,0,3
b,2,1,0,1
c,1,2,0,-1
d,0,3,0,-3


In [30]:
print("Comparisons:")

data = copelands_method_results[1].copy()

data_to_print = {}

for key in data.keys():
  data_to_print[key] = data[key]['comparisons']

df_copelands = pd.DataFrame(data_to_print).T.fillna('-')

df_copelands

Comparisons:


,b,c,d,a
a,win,win,win,-
b,-,win,win,loss
c,loss,-,win,loss
d,loss,loss,-,loss


In [31]:
winner, tree_of_comparisons = parallel_exclusion_method(df)

print("Parallel exclusion method results")
print("Winner:", winner)
print("Tree of comparisons:")
pd.DataFrame(tree_of_comparisons.items(), columns=['Comparison', 'Winner'])

Parallel exclusion method results
Winner: a
Tree of comparisons:


,Comparison,Winner
0,"(a, b)",a
1,"(c, d)",c
2,"(a, c)",a
